In [ ]:
!pip install sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!pip install -q datasets==2.0.0

In [ ]:

!pip install -q seqeval


In [ ]:
!pip install -q pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.11.0+cu113 torchvision==0.9.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#!git clone https://github.com/facebookresearch/detectron2.git
#!python -m pip install -e detectron2

tcmalloc: large alloc 1636958208 bytes == 0x2388000 @  0x7f3f424ae1e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019
tcmalloc: large alloc 1636958208 bytes == 0x63ca8000 @  0x7f3f424ae1e7 0x4a3940 0x5b438c 0x64cfe7 0x59b076 0x515655 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576
tcmalloc: large alloc 1636958208 bytes == 0xc55c8000 @  0x7f3f424ae1e7 0x4a3940 0x59b5e2 0x63a515 0x63bd66 0x63be16 0x59afff 0x515655 0x593dd7 0x511e2c 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595

In [ ]:
##imports
import os
import glob
import json 
import random
from pathlib import Path
from difflib import SequenceMatcher



os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" 
import torch 
import numpy as np
import random
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.use_deterministic_algorithms(True)

import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from PIL import Image
from matplotlib import pyplot, patches
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from transformers import T5Tokenizer, T5EncoderModel, AutoTokenizer

In [ ]:


import torch.nn as nn
from transformers import TrainingArguments, Trainer,LayoutLMv2Model, LayoutLMv2PreTrainedModel
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from dataclasses import dataclass
from dataclasses import fields
from datasets import load_dataset
import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
!rm -r /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/

In [ ]:
# data loading


datasets = load_dataset("darentang/sroie",cache_dir=None)
#datasets = load_dataset("nielsr/funsd",cache_dir=None)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset sroie downloaded and prepared to /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#labels
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)

['O', 'B-COMPANY', 'I-COMPANY', 'B-DATE', 'I-DATE', 'B-ADDRESS', 'I-ADDRESS', 'B-TOTAL', 'I-TOTAL']


In [ ]:

#labels
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id


{'B-ADDRESS': 5,
 'B-COMPANY': 1,
 'B-DATE': 3,
 'B-TOTAL': 7,
 'I-ADDRESS': 6,
 'I-COMPANY': 2,
 'I-DATE': 4,
 'I-TOTAL': 8,
 'O': 0}

In [ ]:
pos2id= {'ADJ': 3,
 'ADP': 13,
 'ADV': 8,
 'AUX': 12,
 'CCONJ': 5,
 'DET': 7,
 'INTJ': 6,
 'NOUN': 15,
 'NUM': 0,
 'PART': 2,
 'PRON': 9,
 'PROPN': 1,
 'PUNCT': 14,
 'SYM': 10,
 'VERB': 4,
 'X': 11,
 'PAD': 19,
 'CONJ':16,
 'SCONJ':17,
 'SPACE':18

 }

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 626
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 347
    })
})

In [ ]:
count = 0
for x in "Life is like a box of chocolates.":
  if x is " ":
    count  += 1
print(count)

6


In [ ]:
dep = ["ACL", "ACOMP", "ADVCL", "ADVMOD", "AGENT", "AMOD", "APPOS", "ATTR", "AUX", "AUXPASS", "CASE", "CC", "CCOMP", "COMPOUND", "CONJ", "CSUBJ", "CSUBJPASS", "DATIVE", "DEP", "DET", "DOBJ", "EXPL", "INTJ", "MARK", "META", "NEG", "NOUNMOD", "NPMOD", "NSUBJ", "NSUBJPASS", "NUMMOD", "OPRD", "PARATAXIS", "PCOMP", "POBJ", "POSS", "PRECONJ", "PREDET", "PREP", "PRT", "PUNCT", "QUANTMOD", "RELCL", "ROOT", "XCOMP","PAD", "COMPLM", "INFMOD", "PARTMOD", "HMOD", "HYPH", "IOBJ", "NUM", "NUMBER", "NMOD", "NN", "NPADVMOD", "POSSESSIVE", "RCMOD", "PAD"]
dep2id = {v:k for k,v in enumerate(dep)}

In [ ]:
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
from transformers import AutoTokenizer
## preprocess the dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer_char = AutoTokenizer.from_pretrained('google/byt5-small')

def char_emb_input_ids( input_ids_batch):
  texts= processor.tokenizer.batch_decode(input_ids_batch)
  model_input_ = tokenizer_char(texts, padding=True, truncation=True).input_ids
  import gc
  gc.collect()

  return model_input_


# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'embedding_char_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
    "pos_tags":  Sequence(feature=Value(dtype='int64')),
    "dep_tags":  Sequence(feature=Value(dtype='int64')),
    
})


def get_tags(input_ids_batch, entype="pos"):
  ## loop for each dcoment
  tags_examples = []
  if entype == "pos":
    tag2id = pos2id
  else:
    tag2id = dep2id

  for input_ids in input_ids_batch:

    y= processor.tokenizer.convert_ids_to_tokens(input_ids)
    words=[]
    for x in y:
      if x.startswith("##"):
        words[-1] = words[-1]+x.replace("##", "").strip()
      else:
        words.append(x)

    text = " ".join(words)
    text = text.replace("[CLS]", "").replace("[SEP]", "").replace("[PAD]", "").strip()
    doc = nlp(text)
    pos_tags = []
    for xx in doc:
      if entype == "pos":
        pos = xx.pos_
        
        pos_tags.append(pos)
      else:
        dep = xx.dep_.upper()
       
        pos_tags.append(dep)
        
        
    count = 0
    total_pos = []
    for each in words:
      if each not in ["[CLS]", "[SEP]", "[PAD]"]:
        total_pos.append(pos_tags[count])
        count += 1
      else:
          total_pos.append("PAD")
    assert len(words) == len(total_pos)

    def convert_to_original_length(y, tags):
   
        count = 0
        r_tags = []
        for index, token in enumerate(y):
            
            if token.startswith("##"):
                r_tags.append(r_tags[-1])
            else:
                r_tags.append(tag2id[tags[count]])
                count += 1
        return r_tags

    tags_examples.append(convert_to_original_length(y, total_pos))
  return tags_examples


    

def preprocess_data(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']
  # input_ids_t5 = tokenizer_t5([" ".join(each) for each in words], padding="max_length", truncation=True).input_ids



  
  
  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True)
  encoded_inputs["embedding_char_ids"] = char_emb_input_ids(encoded_inputs["input_ids"])
  encoded_inputs["dep_tags"] = get_tags(encoded_inputs["input_ids"], entype="dep")
  encoded_inputs["pos_tags"] = get_tags(encoded_inputs["input_ids"])

  return encoded_inputs
  
train_dataset = datasets["train"].map(preprocess_data,batched=True, remove_columns=datasets["train"].column_names,
                                      features=features)
test_dataset = datasets["test"].map(preprocess_data, batched=True, remove_columns=datasets["test"].column_names,
                                       features=features)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [ ]:

##creating a dataloader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4)
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [ ]:
from collections import OrderedDict, UserDict
from typing import Any, BinaryIO, ContextManager, Dict, List, Optional, Tuple, Union

class ModelOutput(OrderedDict):
    """
    Base class for all model outputs as dataclass. Has a `__getitem__` that allows indexing by integer or slice (like a
    tuple) or strings (like a dictionary) that will ignore the `None` attributes. Otherwise behaves like a regular
    python dictionary.
    <Tip warning={true}>
    You can't unpack a `ModelOutput` directly. Use the [`~file_utils.ModelOutput.to_tuple`] method to convert it to a
    tuple before.
    </Tip>
    """

    def __post_init__(self):
        class_fields = fields(self)

        # Safety and consistency checks
        if not len(class_fields):
            raise ValueError(f"{self.__class__.__name__} has no fields.")
        if not all(field.default is None for field in class_fields[1:]):
            raise ValueError(f"{self.__class__.__name__} should not have more than one required field.")

        first_field = getattr(self, class_fields[0].name)
        other_fields_are_none = all(getattr(self, field.name) is None for field in class_fields[1:])

        if other_fields_are_none and not is_tensor(first_field):
            if isinstance(first_field, dict):
                iterator = first_field.items()
                first_field_iterator = True
            else:
                try:
                    iterator = iter(first_field)
                    first_field_iterator = True
                except TypeError:
                    first_field_iterator = False

            # if we provided an iterator as first field and the iterator is a (key, value) iterator
            # set the associated fields
            if first_field_iterator:
                for element in iterator:
                    if (
                        not isinstance(element, (list, tuple))
                        or not len(element) == 2
                        or not isinstance(element[0], str)
                    ):
                        break
                    setattr(self, element[0], element[1])
                    if element[1] is not None:
                        self[element[0]] = element[1]
            elif first_field is not None:
                self[class_fields[0].name] = first_field
        else:
            for field in class_fields:
                v = getattr(self, field.name)
                if v is not None:
                    self[field.name] = v

    def __delitem__(self, *args, **kwargs):
        raise Exception(f"You cannot use ``__delitem__`` on a {self.__class__.__name__} instance.")

    def setdefault(self, *args, **kwargs):
        raise Exception(f"You cannot use ``setdefault`` on a {self.__class__.__name__} instance.")

    def pop(self, *args, **kwargs):
        raise Exception(f"You cannot use ``pop`` on a {self.__class__.__name__} instance.")

    def update(self, *args, **kwargs):
        raise Exception(f"You cannot use ``update`` on a {self.__class__.__name__} instance.")

    def __getitem__(self, k):
        if isinstance(k, str):
            inner_dict = {k: v for (k, v) in self.items()}
            return inner_dict[k]
        else:
            return self.to_tuple()[k]

    def __setattr__(self, name, value):
        if name in self.keys() and value is not None:
            # Don't call self.__setitem__ to avoid recursion errors
            super().__setitem__(name, value)
        super().__setattr__(name, value)

    def __setitem__(self, key, value):
        # Will raise a KeyException if needed
        super().__setitem__(key, value)
        # Don't call self.__setattr__ to avoid recursion errors
        super().__setattr__(key, value)

    def to_tuple(self) -> Tuple[Any]:
        """
        Convert self to a tuple containing all the attributes/keys that are not `None`.
        """
        return tuple(self[k] for k in self.keys())



In [ ]:
device

device(type='cuda')

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


@dataclass
class TokenClassifierOutput(ModelOutput):
    """
    Base class for outputs of token classification models.
    Args:
        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned when `labels` is provided) :
            Classification loss.
        logits (`torch.FloatTensor` of shape `(batch_size, sequence_length, config.num_labels)`):
            Classification scores (before SoftMax).
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer) of
            shape `(batch_size, sequence_length, hidden_size)`.
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """

    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None






class LayoutLMv2ForTokenClassification2(LayoutLMv2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.layoutlmv2 = LayoutLMv2Model.from_pretrained("microsoft/layoutlmv2-base-uncased", output_hidden_states=True)  
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.embedding_dep= nn.Embedding(60, 50, max_norm=True)
        self.embedding_pos= nn.Embedding(21, 50, max_norm=True)
        self.model_char = T5EncoderModel.from_pretrained('google/byt5-small')
        self.model_char.to(device)
        
        # print(self.charemb.model_char(train_dataset[0]["embedding_char_ids"].to(device)))

        ## for without lstm
        #self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        #self.lstm1 =  nn.LSTM(input_size=768, hidden_size=256, batch_first=True, bidirectional=True)
        #self.lstm2 =  nn.LSTM(input_size=256*2, hidden_size=256, batch_first=True, bidirectional=True)
        self.lstm3 =  nn.GRU(input_size=768+736+50+50, hidden_size=128, batch_first=True, bidirectional=True)
        self.lstm4 =  nn.GRU(input_size=128*2, hidden_size=128, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(128*2, config.num_labels)

        # Initialize weights and apply final processing
        #self.post_init()

    def get_input_embeddings(self):
        return self.layoutlmv2.embeddings.word_embeddings
        
    def char_emb_to_word(self,model_inputs):
      ## loop for each document
      total_emb = []
      
      with torch.no_grad(): 
          # print(torch.tensor([[2,3]]).shape)
          # print(model_inputs.shape)
          embeddings_chars = self.model_char(model_inputs)["last_hidden_state"]
          # print(model_inputs.shape)
          for i, embeddings_char in enumerate(embeddings_chars):
            spaces = (model_inputs[i] == 35).nonzero().flatten()
            truc = int(embeddings_char.shape[-1]/2)
            final = torch.zeros((512, truc))
            count = 0
            index = 0
            for sp in spaces:
              final[index,:] = torch.mean(embeddings_char[count:sp, :truc], 0)
              count  = sp+1
              index += 1
            total_emb.append(final)
      return torch.stack(total_emb, 0)

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        
        embedding_char_ids: Optional[torch.Tensor] = None,
        pos_tags: Optional[torch.LongTensor] = None,
        dep_tags: Optional[torch.LongTensor] = None,
        bbox: Optional[torch.LongTensor] = None,
        image: Optional[torch.FloatTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        Returns:
        Examples:
        ```python
        >>> from transformers import LayoutLMv2Processor, LayoutLMv2ForTokenClassification
        >>> from PIL import Image
        >>> processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
        >>> model = LayoutLMv2ForTokenClassification.from_pretrained("microsoft/layoutlmv2-base-uncased")
        >>> image = Image.open("name_of_your_document - can be a png file, pdf, etc.").convert("RGB")
        >>> words = ["hello", "world"]
        >>> boxes = [[1, 2, 3, 4], [5, 6, 7, 8]]  # make sure to normalize your bounding boxes
        >>> word_labels = [0, 1]
        >>> encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="pt")
        >>> outputs = model(**encoding)
        >>> loss = outputs.loss
        >>> logits = outputs.logits
        ```"""

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        # print(embedding_char_ids.shape)
        char_emb =  self.char_emb_to_word(embedding_char_ids.to(device))
        
        dep_output = self.embedding_dep(dep_tags)
        pos_output = self.embedding_pos(pos_tags)
        total_emb = torch.cat([dep_output,pos_output],2)
      
      
        # if torch.isnan(char_emb).any():
        #   print(embedding_char_ids)
        
        pos_output = torch.cat((char_emb.to(device), total_emb), 2)
        
        # print(pos_output.shape)
        outputs = self.layoutlmv2(
            input_ids=input_ids,
            bbox=bbox,
            image=image,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
       
        sequence_output = torch.stack(list(outputs[2]), dim=0)
        # only take the text part of the output representations
        
        sequence_output=torch.mean(sequence_output,0)[:, :seq_length]
        # print(sequence_output.shape)
        # print(pos_output.shape)
        sequence_output = torch.cat([sequence_output, pos_output], -1)
        # sequence_output = torch.mean(torch.stack([sequence_output,pos_output], dim=0),0)
        
        # sequence_output = self.dropout(sequence_output)
        sequence_output, _ = self.lstm3(sequence_output)
        # print(sequence_output.shape)
       
        sequence_output, _ = self.lstm4(sequence_output)
       
        
        sequence_output = self.dropout(sequence_output)
        #print(sequence_output.shape)

        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
torch.cuda.empty_cache()


In [ ]:
device

device(type='cuda')

In [ ]:

from datasets import load_metric
import json
from transformers import get_scheduler

model = LayoutLMv2PreTrainedModel.from_pretrained("microsoft/layoutlmv2-base-uncased")
model.config.id2label = id2label
model.config.label2id = label2id
model.config.num_labels = len(id2label)
models = LayoutLMv2ForTokenClassification2(model.config)

for p in models.model_char.parameters():
  p.requires_grad = False
for p in models.layoutlmv2.parameters():
  p.requires_grad = False

# Set id2label and label2id 

# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return json.dump({
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        })



class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
      return train_dataloader

    def get_test_dataloader(self, test_dataset):
      return test_dataloader

from transformers.optimization import Adafactor, AdafactorSchedule, AdamW
#optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

optimizer = AdamW(params=models.parameters(), lr=0.008)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=2000,
)
args = TrainingArguments(
    
    output_dir="layoutlmv2-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=2000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.4, # we warmup a bit
    
)

# Initialize our Trainer
trainer = FunsdTrainer(
    model=models,
    args=args,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler)
)


loading configuration file https://huggingface.co/microsoft/layoutlmv2-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/882f0cab8dbb456e5b1d6e3b96e864be0cb6c2bc5d20ee88eeda10b3c0317332.a3f80b6502f00efe74a01c6a007f196803229a24224659c81c1da7c4cf5316e8
Model config LayoutLMv2Config {
  "attention_probs_dropout_prob": 0.1,
  "convert_sync_batchnorm": true,
  "coordinate_size": 128,
  "detectron2_config_args": {
    "MODEL.ANCHOR_GENERATOR.SIZES": [
      [
        32
      ],
      [
        64
      ],
      [
        128
      ],
      [
        256
      ],
      [
        512
      ]
    ],
    "MODEL.BACKBONE.NAME": "build_resnet_fpn_backbone",
    "MODEL.FPN.IN_FEATURES": [
      "res2",
      "res3",
      "res4",
      "res5"
    ],
    "MODEL.MASK_ON": true,
    "MODEL.PIXEL_STD": [
      57.375,
      57.12,
      58.395
    ],
    "MODEL.POST_NMS_TOPK_TEST": 1000,
    "MODEL.RESNETS.ASPECT_RATIOS": [
      [
        0.5,
        1.0,
    

In [ ]:
total_params = sum(p.numel() for p in models.parameters() if p.requires_grad)

In [ ]:
total_params

1634523

In [ ]:
trainer.train()
# predictions, labels, metrics = trainer.predict(test_dataset)

***** Running training *****
  Num examples = 626
  Num Epochs = 13
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2000


Step,Training Loss
500,0.073400


Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-500
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/pytorch_model.bin


Step,Training Loss
500,0.073400
1000,0.019200
1500,0.008500
2000,0.002700


Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-1000
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1000/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-1500
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1500/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-2000
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-2000/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2000, training_loss=0.025929803729057312, metrics={'train_runtime': 2824.7953, 'train_samples_per_second': 5.664, 'train_steps_per_second': 0.708, 'total_flos': 9664787726524416.0, 'train_loss': 0.025929803729057312, 'epoch': 12.74})

In [ ]:
torch.cuda.empty_cache()

In [ ]:

from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)

In [ ]:



eval_loss = 0.0
nb_eval_steps = 0
preds = None
out_label_ids = None

# put model in evaluation mode
models.eval()
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        print(batch.keys())
        input_ids = batch["input_ids"].to(device)
        bbox = batch["bbox"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        image = batch['image'].to(device)
        pos_tags = batch['pos_tags'].to(device)
        dep_tags = batch['pos_tags'].to(device)

        input_ids_char = batch["embedding_char_ids"].to(device)

        # forward pass
        outputs = models( embedding_char_ids =input_ids_char, pos_tags = pos_tags,dep_tags = dep_tags,image = image , input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
        # get the loss and logits
        tmp_eval_loss = outputs.loss
        logits = outputs.logits

        eval_loss += tmp_eval_loss.item()
        nb_eval_steps += 1

        # compute the predictions
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, labels.detach().cpu().numpy(), axis=0
            )

# compute average evaluation loss
eval_loss = eval_loss / nb_eval_steps
preds = np.argmax(preds, axis=2)

out_label_list = [[] for _ in range(out_label_ids.shape[0])]
preds_list = [[] for _ in range(out_label_ids.shape[0])]

for i in range(out_label_ids.shape[0]):
    for j in range(out_label_ids.shape[1]):
        if out_label_ids[i, j] != -100:
            out_label_list[i].append(id2label[out_label_ids[i][j]])
            preds_list[i].append(id2label[preds[i][j]])

results = {
    "loss": eval_loss,
    "precision": precision_score(out_label_list, preds_list),
    "recall": recall_score(out_label_list, preds_list),
    "f1": f1_score(out_label_list, preds_list),
}
print(results)

Evaluating:   0%|          | 0/174 [00:00<?, ?it/s]

dict_keys(['image', 'input_ids', 'embedding_char_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels', 'pos_tags', 'dep_tags'])


Evaluating:   1%|          | 1/174 [00:00<01:58,  1.45it/s]

dict_keys(['image', 'input_ids', 'embedding_char_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels', 'pos_tags', 'dep_tags'])


Evaluating:   1%|          | 2/174 [00:01<01:57,  1.47it/s]

...

dict_keys(['image', 'input_ids', 'embedding_char_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels', 'pos_tags', 'dep_tags'])


Evaluating:  99%|█████████▉| 173/174 [01:57<00:00,  1.47it/s]

dict_keys(['image', 'input_ids', 'embedding_char_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels', 'pos_tags', 'dep_tags'])


Evaluating: 100%|██████████| 174/174 [01:58<00:00,  1.47it/s]


{'loss': 0.03087681388053337, 'precision': 0.9353448275862069, 'recall': 0.9380403458213257, 'f1': 0.9366906474820144}
